In [3]:
    import requests
    import pandas as pd
    from datetime import date

    # URL template (corrected)
    url = "https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15"

    # Instrument tokens
    tickerData = {"NIFTY": {"id": 256265}, "BANKNIFTY": {"id": 260105}}

    # Interval mapping
    KI = {
        "3m": "3minute",
        "5m": "5minute",
        "15m": "15minute",
        "60m": "60minute",
        "day": "day",
    }

    # Function to get candle data
    def getCandles(symbol, fromDate, toDate, timeframe):
        headers = {
            "authorization": "enctoken l9RBs2mUaCOtHh76/Al766xp7yJ1ndunq9zE2PslZvpMEXu0ccRXlWhWHNUOyJX19msbiNnAcd37PdO0yb1ikRiT3q1N7v2hutAK2FB3s2GAGxkfVgusyQ=="
            # Replace this with your actual token securely
        }
        curUrl = url.format(tickerData[symbol]["id"], timeframe, fromDate, toDate)
        print("Requesting:", curUrl)

        try:
            session = requests.session()
            response = session.get(curUrl, headers=headers)
            r = response.json()
        except Exception as error:
            print("API request failed:", error)
            return pd.DataFrame()

        try:
            history = r["data"]["candles"]
            history_df = pd.DataFrame(history)
            history_df = history_df.rename(
                columns={
                    0: "Datetime",
                    1: "open",
                    2: "high",
                    3: "low",
                    4: "close",
                    5: "volume",
                }
            )
            history_df["Datetime"] = pd.to_datetime(history_df["Datetime"])
            history_df.set_index("Datetime", inplace=True)
            return history_df
        except Exception as error:
            print("Data processing failed:", error)
            return pd.DataFrame()

    # === Hammer Candlestick Pattern Detector ===
    def stochastic(df_dict, lookback=14, k=3, d=3):
        """
        Function to calculate the Stochastic Oscillator.
        %K = (Close - Lowest Low) / (Highest High - Lowest Low) * 100
        %D = SMA of %K
        """
        df = df_dict.copy()

        df["HH"] = df["high"].rolling(lookback).max()
        df["LL"] = df["low"].rolling(lookback).min()

        df["%K"] = 100 * (df["close"] - df["LL"]) / (df["HH"] - df["LL"])
        df["%K"] = df["%K"].rolling(k).mean()
        df["%D"] = df["%K"].rolling(d).mean()

        df.drop(["HH", "LL"], axis=1, inplace=True)
        return df

        

    print("Start")
    symbol_data = getCandles("NIFTY", "2025-03-20", "2025-03-25", KI["15m"])


    if not symbol_data.empty:
        stochastic_df = stochastic(symbol_data)
        print(stochastic_df.tail(40))
    else:
        print("No OHLC data available.")


Start
Requesting: https://kite.zerodha.com/oms/instruments/historical/256265/5minute?user_id=HHS112&oi=1&from=2025-06-11&to=2025-06-15
                               open      high       low     close  volume  6  \
Datetime                                                                       
2025-06-13 12:10:00+05:30  24685.45  24692.75  24683.35  24688.95       0  0   
2025-06-13 12:15:00+05:30  24688.80  24724.60  24688.80  24723.95       0  0   
2025-06-13 12:20:00+05:30  24722.75  24723.90  24709.00  24710.35       0  0   
2025-06-13 12:25:00+05:30  24709.20  24712.85  24703.85  24706.45       0  0   
2025-06-13 12:30:00+05:30  24704.35  24711.05  24696.10  24696.10       0  0   
2025-06-13 12:35:00+05:30  24695.10  24698.80  24690.85  24692.40       0  0   
2025-06-13 12:40:00+05:30  24692.05  24696.45  24687.45  24690.80       0  0   
2025-06-13 12:45:00+05:30  24688.50  24695.80  24681.45  24682.40       0  0   
2025-06-13 12:50:00+05:30  24680.20  24687.75  24675.90  24683.30